In [2]:
from engines.engine_model import EngineProp
from build_rocket import RocketModel
import numpy as np
import time
import joblib
from rocketcea.cea_obj_w_units import CEA_Obj


# Comparando tempo de execução do modelo de motor
## Metodologia:
 - Calcular sequencialmente o tempo de execução para 10.000 indivíduos considerando o CEA e o reg_model

### 1. CEA

In [3]:
# Inicializando parâmetros de entrada
MR = 2.8
Pc = 11.5 * 1e6
eps = 180

# Inicializando CEA e arvore de decisao 
reg_path = '/home/ubuntu/Mestrado/modelo_foguete/model/engines/decision_tree_model.pkl'
reg_model = joblib.load(reg_path)
#reg_model = False
cea_obj = ceaObj = CEA_Obj( oxName='LOX', fuelName='RP-1', pressure_units='MPa', cstar_units='m/s', temperature_units='K')


In [15]:
time_start = time.time()
for i in range(10000):
    engine_cea = EngineProp(
        MR=MR,
        Pc=Pc,
        eps=eps,
        nozzleDiam=0.23125,
        cea_obj=cea_obj,
        verbose=False
    )
    engine_cea.estimate_all()
time_end = time.time()
tempo_engine_cea = time_end - time_start
engine_cea.print_all_parameters()
print("#" * 40)
print(f"Tempo total de execução: {tempo_engine_cea}")


Isp Vac (s): 372.0353983280371
Isp Sea (s): 209.75868815824597
Mass flow (kg/s): 310.1185528455126
Thrust Vac (kN): 1295.4477569099522
Thrust Sea (kN): 539.4406654083928
########################################
Tempo total de execução: 15.520617723464966


### 2. Árvore de Decisão

In [14]:
time_start = time.time()
for i in range(10000):
    engine_arvore = EngineProp(
        MR=MR,
        Pc=Pc,
        eps=eps,
        nozzleDiam=0.23125,
        cea_obj=False,
        verbose=False,
        reg_model=reg_model
    )
    engine_arvore.estimate_all()
time_end = time.time()
tempo_engine_arvore = time_end - time_start
engine_arvore.print_all_parameters()
print("#" * 40)
print(f"Tempo total de execução: {tempo_engine_arvore}")


Isp Vac (s): 371.0670875636104
Isp Sea (s): 207.56445875314893
Mass flow (kg/s): 312.410473845755
Thrust Vac (kN): 1293.2092411411538
Thrust Sea (kN): 537.2021496395942
########################################
Tempo total de execução: 5.927952766418457


In [18]:
print("Melhora no tempo de execução:")
ratio = tempo_engine_cea/tempo_engine_arvore
print(f"Árvore de decisão é {ratio} x mais rápida")

Melhora no tempo de execução:
Árvore de decisão é 2.618208736646563 x mais rápida


# Comparando o tempo de execução do modelo completo

### 1. CEA

In [20]:
# Inicializando parâmetros:
engineParams = {"oxName": "LOX",
                "fuelName": "RP-1",
                "combPressure": 11.5 * 1e6,
                "MR": 2.8,
                "nozzleDiam": 0.23125,
                "eps": 180}

engineParamsFirst = {"oxName": "LOX",
                "fuelName": "RP-1",
                "combPressure": 11.5 * 1e6,
                "MR": 2.8,
                "nozzleDiam": 0.23125,
                "eps": 25}

payloadBayParams = {"payloadHeight": 6.7,
                    "payloadRadius": 4.6/2,
                    "payloadMass": 7500,
                    "lowerStageRadius": 2.1,
                    "lowerRocketSurfaceArea": 0} # 0 porque ainda nao temos esse valor

upperStageStructureParams = {"oxName": "LOX",
                                "fuelName": "RP1",
                                "MR": 2.8,
                                "tankPressure": 0.1,
                                "radius": 2.1,
                            } # 0 porque ainda nao temos esse valor
lowerStageStructureParams = {"oxName": "LOX",
                            "fuelName": "RP1",
                            "MR": 2.8,
                            "tankPressure": 0.1,
                            "radius": 2.8,
                        } # 0 porque ainda nao temos esse valor


In [24]:
time_start = time.time()
for i in range(10000):
    rocket_model_cea = RocketModel(upperEngineParams=engineParams,
                            firstEngineParams=engineParamsFirst,
                            payloadBayParams=payloadBayParams,
                            upperStageStructureParams=upperStageStructureParams,
                            firstStageStructureParams = lowerStageStructureParams,
                            deltaV_upperStage=9000,
                            deltaV_landing=2000,
                            deltaV_firstStage=3000,
                            nEnginesUpperStage=1,
                            nEnignesFirstStage=9,
                            reg_model=False,
                            cea_obj=cea_obj,
                            )

    rocket_model_cea.build_all()
    #rocket_model.print_all_parameters()
time_end = time.time()
tempo_rocket_cea = time_end - time_start
print("#" * 40)
print(f"Tempo total de execução: {tempo_rocket_cea}")



########################################
Tempo total de execução: 36.98487210273743


### 2. Árvore de Decisão

In [25]:
time_start = time.time()
for i in range(10000):
    rocket_model_arvore = RocketModel(upperEngineParams=engineParams,
                            firstEngineParams=engineParamsFirst,
                            payloadBayParams=payloadBayParams,
                            upperStageStructureParams=upperStageStructureParams,
                            firstStageStructureParams = lowerStageStructureParams,
                            deltaV_upperStage=9000,
                            deltaV_landing=2000,
                            deltaV_firstStage=3000,
                            nEnginesUpperStage=1,
                            nEnignesFirstStage=9,
                            reg_model=reg_model,
                            cea_obj=False,
                            )

    rocket_model_arvore.build_all()
    #rocket_model.print_all_parameters()
time_end = time.time()
tempo_rocket_arvore = time_end - time_start
print("#" * 40)
print(f"Tempo total de execução: {tempo_rocket_arvore}")


########################################
Tempo total de execução: 15.292006015777588


In [26]:
print("Melhora no tempo de execução:")
ratio = tempo_rocket_cea/tempo_rocket_arvore
print(f"Árvore de decisão é {ratio} x mais rápida")

Melhora no tempo de execução:
Árvore de decisão é 2.418575565859583 x mais rápida
